In [ ]:

import pandas as pd
import re
import numpy as np
import nltk.corpus

nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
import nltk 
nltk.download('punkt') 
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import spacy
from string import digits
from nltk.tokenize import sent_tokenize, word_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from collections import Counter
import seaborn as sns

In [ ]:
def open_file(namefile):
    with open(namefile+'.txt','r') as f:
        data=f.readlines()  
    return data

In [ ]:
def to_df(data):
     df = pd.DataFrame(columns=['text'],data = data)
     return df

In [ ]:
def clean_text(df):
    df['text_clean'] = df['text'].str.lower()
    df['text_clean'] = df['text_clean'].apply(lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem))  
    # remove numbers
    df['text_clean'] = df['text_clean'].apply(lambda elem: re.sub(r"\d+", "", elem))
    return df

In [ ]:
def bigram_5gram(df):
    vec = CountVectorizer(ngram_range = (2, 5),stop_words='english')
    list_= list()
    list_=df['text_clean'].tolist()
    X_train = vec.fit_transform(list_)
    bigram_df=pd.DataFrame(X_train.toarray(), columns=vec.get_feature_names())
    return bigram_df

In [ ]:
def onegram(df):
    vec = CountVectorizer(ngram_range = (1, 1),stop_words='english')
    list_= list()
    list_=df['text_clean'].tolist()
    X_train1 = vec.fit_transform(list_)
    df_c=pd.DataFrame(X_train1.toarray(), columns=vec.get_feature_names())
    return df_c

In [ ]:
def most_common_to_list(df):
    listcount=list()
    j=0
    for val in df:
        listcount.append([val,0])
        for i in range(len(df[val])):
            listcount[j][1] +=df[val][i]
        j+=1
        
    listcount.sort(key=lambda y: y[1],reverse= True)
    return listcount

In [ ]:
def mostcommon_to_file(listcount,file_name,flag):
    if flag==0:
         textfile = open(file_name+",most common.txt", "w")
    elif flag ==1:
        textfile = open(file_name+",most common 2 to 5 gram.txt", "w")
    
    for i in range(len(listcount)):
        textfile.write(listcount[i][0])
        textfile.write(' ')
        textfile.write(str(listcount[i][1]))
        textfile.write(', ')
    print("The file is ready for: "+file_name)

In [ ]:
def mostcommon_to_csv(listcount,file_name,flag):
    df=pd.DataFrame(columns=["common_word","count"])
    list1=[]
    list2=[]
    for i in range(len(listcount)):
        list1.append(listcount[i][0])
        list2.append(listcount[i][1])
    df["common_word"]=list1
    df["count"]=list2
    if flag==0:
        df.to_csv(file_name+",most common.csv")
    elif flag ==1:
        df.to_csv(file_name+",most common 2 to 5 gram.csv")
    print("The csv is ready for: "+file_name)


In [ ]:
def into_barplot(listcount):
    word = []
    frequency = []
    plt.figure(figsize=(150,20))
    print(len(listcount))
    j=0
    for i in range(len(listcount)):
        if listcount[i][1] > 50 and j<15:
            word.append(listcount[i][0])
            frequency.append(listcount[i][1])
            j+=1

    sns.barplot(y= word, x=frequency, color='b')
    plt.show()

In [ ]:
def into_wordcloud(listcount):
    text = "".join(str(title) for i in range(len(listcount)) for title in listcount[i][0]+', ' )
    wordcloud = WordCloud().generate(text)

    # Display the generated image:
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()

In [ ]:
file_names=['[Martinus Nijhoff Philosophy Texts 2] J. N. Mohanty (auth.), William McKenna, Robert M. Harlan, Laurence E. Winters (eds.) - Apriori and World_ European Contributions',
"066_Embree and Nenon. (2013) Husserls Ideen",
"Dilthey - The Formation of the Historical World in the Human Sciences",
"VI-Dilthey Selected Works, vol. 6 - Ethical and World-View Philosophy"]
for name in file_names:
    data= open_file(name)
    df = to_df(data)
    df=clean_text(df)
    bigram_df=bigram_5gram(df)
    onegram_df=onegram(df)
    listcount=most_common_to_list(onegram_df)
    mostcommon_to_file(listcount,name,0)
    mostcommon_to_csv(listcount,name,0)
    listcountbi=most_common_to_list(bigram_df)
    mostcommon_to_file(listcountbi,name,1)
    # mostcommon_to_csv(listcountbi,name,1)
    into_barplot(listcount)
    into_wordcloud(listcount)
    
    